In [1]:
import tensorflow as tf
#import tensorflow.examples.tutorials.mnist.input_data as input_data
#mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [14]:
'''prepare data'''

num_class = 10

# data
train_dict = h5py.File("/home/mmt/workspace/DATA_SET/KMNIST_0430/train_x.mat", 'r')
data = np.array(train_dict["data"]).astype(np.float32).transpose([2,1,0])
data = np.reshape(data[0:9000,:,:],[-1,1024])

data_label = h5py.File("/home/mmt/workspace/DATA_SET/KMNIST_0430/train_y.mat", 'r')
data_label = np.array(data_label["train_y"]).astype(np.int32).transpose([1,0])
data_label = np.eye(num_class)[np.squeeze(data_label[0:9000,:]).reshape(-1)].astype(np.float32)

data = np.concatenate((data, data_label), axis = 1)

print(data.shape, data_label.shape)

test_dict = h5py.File("/home/mmt/workspace/DATA_SET/KMNIST_0430/test_x.mat", 'r')
data_test = np.array(test_dict["data"]).astype(np.float32).transpose([2,1,0])
data_test = np.reshape(data_test[0:1500,:,:],[-1,1024])

data_test_label = h5py.File("/home/mmt/workspace/DATA_SET/KMNIST_0430/test_y.mat", 'r')
data_test_label = np.array(data_test_label["test_y"]).astype(np.int32).reshape(-1)

data_test_label = data_test_label[5::6]

data_test_label = np.eye(num_class)[np.squeeze(data_test_label[0:1500]).reshape(-1)].astype(np.float32)
data_test = np.concatenate((data_test, data_test_label), axis = 1)

print(data_test.shape, data_test_label.shape)

print("Load data done.")

(9000, 1034) (9000, 10)
(1500, 1034) (1500, 10)
Load data done.


In [15]:
print ('data_train_shape:',np.shape(data))
print ('data_test_shape:',np.shape(data_test))

total_img = np.shape(data)[0]
print ('total_img:',total_img)
#plt.imshow(data[1000,:1024].reshape([32,32]))
#plt.show()

np.random.shuffle(data)
import pickle
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.framework import ops

def weight_variable(shape):
  initial =  tf.random.truncated_normal(shape, stddev=0.4)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)
  
def binarize(x):
    """
    Clip and binarize tensor using the straight through estimator (STE) for the gradient.
    """
    g = tf.compat.v1.get_default_graph()

    with ops.name_scope("Binarized") as name:
        #x=tf.clip_by_value(x,-1,1)
        with g.gradient_override_map({"Sign": "Identity"}):
            return tf.sign(x)

data_train_shape: (9000, 1034)
data_test_shape: (1500, 1034)
total_img: 9000


In [16]:
with tf.device('/gpu:1'):
  gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
  sess = tf.compat.v1.InteractiveSession(
              config=tf.compat.v1.ConfigProto(
              log_device_placement=False,
              allow_soft_placement=True,
              gpu_options=gpu_options,
              ))
  ckpt_dir = './BWN_tf_ckpt'
  restore = True
  Train = True

  if Train==True:
    restore = False


    
  x = tf.compat.v1.placeholder("float", shape=[None, 1024])
  y_ = tf.compat.v1.placeholder("float", shape=[None, 10])

  W_fc1 = weight_variable([1024, 16])
  W_fc1 = tf.clip_by_value(W_fc1,-256,256)
  bin_W_fc1 = binarize(W_fc1)
  #b_fc1 = bias_variable([16])
  #h_fc1 = tf.matmul(x, bin_W_fc1) + b_fc1
  h_fc1 = tf.matmul(x, bin_W_fc1)
  h_fc1 = tf.clip_by_value(h_fc1,-256,256)
  bin_h_fc1 = binarize(h_fc1)
  W_fc2 = weight_variable([16, 10])
  #W_fc2 = tf.clip_by_value(W_fc2,-1.5,1.5)
  #bin_W_fc2 = binarize(W_fc2)
  y = tf.nn.softmax(tf.matmul(bin_h_fc1,W_fc2))

  lr_decay_step=10000
  start_lr=0.01
  global_steps = tf.Variable(0,trainable=False)

  cross_entropy = -tf.reduce_sum(y_*tf.math.log(y))
  lr = tf.compat.v1.train.exponential_decay(start_lr, global_steps, lr_decay_step, 0.2, staircase=True)
  train_step = tf.compat.v1.train.AdamOptimizer(learning_rate=lr).minimize(cross_entropy,global_step=global_steps)
  correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
  saver = tf.compat.v1.train.Saver()

  max_abs_sum = 200
  max_abs_sum_pre = 400
  img_index = 0
  batch_num = 1024
  max_test_acc = 0
  max_train_acc = 0
  while(max_abs_sum>10):

    sess.run(tf.compat.v1.global_variables_initializer())
    #save_path = saver.save(sess,"./model.ckpt")
    if restore==True:
      ckpt_model = tf.train.latest_checkpoint(ckpt_dir)
      saver.restore(sess, ckpt_model)
      print ('Model restore done!')


    if Train==True:
          for i in range(10000):
        if img_index+batch_num>=total_img:
          np.random.shuffle(data)
          img_index=0
        train_img = data[img_index:(img_index+batch_num),:1024]
        train_label = data[img_index:(img_index+batch_num),1024:]
        img_index+=batch_num
          
        
        

        if i%200 == 0:
          train_accuracy = accuracy.eval(feed_dict={
              x: data[:,:1024], y_: data[:,1024:]})
          test_accuracy = accuracy.eval(feed_dict={
              x:data_test[:,:1024], y_: data_test[:,1024:]})
          if test_accuracy > max_test_acc: max_test_acc = test_accuracy
          if train_accuracy > max_train_acc: max_train_acc = train_accuracy
          print ("step %d, training accuracy %g, test accuracy %g, learning rate %g, max_train_acc %g, max_test_acc %g"%(i, train_accuracy,test_accuracy,sess.run(lr), max_train_acc, max_test_acc))
          
        train_step.run(feed_dict={x: train_img, y_: train_label})

      saver.save(sess, ckpt_dir+'/ckpt',global_step=i)
      print ('Model save done!')

    acc = accuracy.eval(feed_dict={x: data[:,:1024], y_: data[:,1024:]})
    print ("train accuracy %g"%(acc))

    acc_test = accuracy.eval(feed_dict={x: data_test[:,:1024], y_: data_test[:,1024:]})
    print ("test accuracy %g, max_test_acc %g"%(acc_test, max_test_acc))
    '''
    f=open('noise_mismatch_result.txt','a')
    #print (np.shape(mnist.test.images))
    for noise_var in range(-20, 60, 1):
      ran_noise = np.random.normal(0,1.0/pow(10,noise_var/2.0),[10000,784])
      ran_mismatch = np.random.normal(0,0.034,[10000,784])
      images_scaled = (mnist.test.images+ran_noise)*(1+ran_mismatch)
      acc = accuracy.eval(feed_dict={x: images_scaled, y_: mnist.test.labels})
      print ("noise_var: %d dB, mismatch: 0.034, test accuracy %g"%(noise_var,acc))
      f.writelines(str(acc)+'\n')

    for mismatch_var in range(1,1000): 
      ran_noise = np.random.normal(0,1.0/pow(10,40/2.0),[10000,784])
      ran_mismatch = np.random.normal(0,0.001*mismatch_var,[10000,784])
      images_scaled = (mnist.test.images+ran_noise)*(1+ran_mismatch)
      acc = accuracy.eval(feed_dict={x: images_scaled, y_: mnist.test.labels})
      print ("noise_var: 40 dB, mismatch: %.3f, test accuracy %g"%(mismatch_var,acc))
      f.writelines(str(acc)+'\n')
    #np.save('./model.npy',sess.run(bin_W_fc1))
    '''
    weight=sess.run(bin_W_fc1)

    all_sum = np.zeros(16)
    for i in range(16):
      all_sum[i] = np.sum(weight[:,i])
      print (all_sum[i])
    max_abs_sum = np.max(abs(all_sum))
    print ('max_abs_sum:',max_abs_sum)
    if max_abs_sum<max_abs_sum_pre:
      np.save('./model_bin_fc1_'+str(int(max_abs_sum))+'_'+str(int(acc*10000))+'_maxtest'+str(int(max_test_acc*10000))+'%.npy',weight)
      np.save('./model_fc2_'+str(int(max_abs_sum))+'_'+str(int(acc*10000))+'%.npy',sess.run(W_fc2))
      max_abs_sum_pre=max_abs_sum

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
step 0, training accuracy 0.146333, test accuracy 0.139333, learning rate 0.01, max_train_acc 0, max_test_acc 0
step 200, training accuracy 0.701333, test accuracy 0.554, learning rate 0.01, max_train_acc 0.146333, max_test_acc 0.139333
step 400, training accuracy 0.747444, test accuracy 0.568, learning rate 0.01, max_train_acc 0.701333, max_test_acc 0.554
step 600, training accuracy 0.764333, test accuracy 0.586667, learning rate 0.01, max_train_acc 0.747444, max_test_acc 0.568
step 800, training accuracy 0.771, test accuracy 0.582, learning rate 0.01, max_train_acc 0.764333, max_test_acc 0.586667
step 1000, training accuracy 0.759333, test accuracy 0.592, learning rate 0.01, max_train_acc 0.771, max_test_acc 0.586667
step 1200, training accuracy 0.761444, test accuracy 0.576667, learning rate 0.01, max_train_acc 0.771, max_test_acc 0.592
step 1400, training accuracy 0.790444, test accuracy 0

step 2400, training accuracy 0.809778, test accuracy 0.581333, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 2600, training accuracy 0.821444, test accuracy 0.596, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 2800, training accuracy 0.816, test accuracy 0.577333, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 3000, training accuracy 0.828222, test accuracy 0.615333, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 3200, training accuracy 0.827333, test accuracy 0.589333, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 3400, training accuracy 0.824333, test accuracy 0.580667, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 3600, training accuracy 0.821667, test accuracy 0.572, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 3800, training accuracy 0.821556, test accuracy 0.580667, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 4000

step 5400, training accuracy 0.783778, test accuracy 0.562, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 5600, training accuracy 0.796778, test accuracy 0.599333, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 5800, training accuracy 0.813556, test accuracy 0.596667, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 6000, training accuracy 0.816111, test accuracy 0.606667, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 6200, training accuracy 0.780556, test accuracy 0.571333, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 6400, training accuracy 0.824111, test accuracy 0.612667, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 6600, training accuracy 0.817667, test accuracy 0.62, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 6800, training accuracy 0.803333, test accuracy 0.595333, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 70

step 8400, training accuracy 0.745556, test accuracy 0.502, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 8600, training accuracy 0.742, test accuracy 0.550667, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 8800, training accuracy 0.780889, test accuracy 0.564, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 9000, training accuracy 0.755444, test accuracy 0.522667, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 9200, training accuracy 0.754667, test accuracy 0.565333, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 9400, training accuracy 0.782667, test accuracy 0.572667, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 9600, training accuracy 0.739889, test accuracy 0.540667, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
step 9800, training accuracy 0.765111, test accuracy 0.538667, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.62
Model sav

step 1000, training accuracy 0.696111, test accuracy 0.511333, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.620667
step 1200, training accuracy 0.754222, test accuracy 0.574667, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.620667
step 1400, training accuracy 0.765556, test accuracy 0.576, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.620667
step 1600, training accuracy 0.745556, test accuracy 0.547333, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.620667
step 1800, training accuracy 0.750667, test accuracy 0.549333, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.620667
step 2000, training accuracy 0.774222, test accuracy 0.563333, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.620667
step 2200, training accuracy 0.764222, test accuracy 0.545333, learning rate 0.01, max_train_acc 0.845778, max_test_acc 0.620667
step 2400, training accuracy 0.784667, test accuracy 0.584667, learning rate 0.01, max_train_acc 0.8

...

step 5400, training accuracy 0.732778, test accuracy 0.538667, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 5600, training accuracy 0.714333, test accuracy 0.526667, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 5800, training accuracy 0.675889, test accuracy 0.479333, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 6000, training accuracy 0.687667, test accuracy 0.520667, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 6200, training accuracy 0.701111, test accuracy 0.510667, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 6400, training accuracy 0.705333, test accuracy 0.514, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 6600, training accuracy 0.702667, test accuracy 0.506667, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 6800, training accuracy 0.731, test accuracy 0.550667, learning rate 0.01, max_train_acc 0.8637

step 8000, training accuracy 0.777222, test accuracy 0.549333, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 8200, training accuracy 0.773556, test accuracy 0.554667, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 8400, training accuracy 0.774111, test accuracy 0.56, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 8600, training accuracy 0.797667, test accuracy 0.569333, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 8800, training accuracy 0.758778, test accuracy 0.564667, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 9000, training accuracy 0.784556, test accuracy 0.579333, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 9200, training accuracy 0.783556, test accuracy 0.564, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 9400, training accuracy 0.777222, test accuracy 0.568, learning rate 0.01, max_train_acc 0.863778, 

step 400, training accuracy 0.679556, test accuracy 0.509333, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 600, training accuracy 0.701778, test accuracy 0.518667, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 800, training accuracy 0.712444, test accuracy 0.506, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 1000, training accuracy 0.715667, test accuracy 0.523333, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 1200, training accuracy 0.735222, test accuracy 0.542667, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 1400, training accuracy 0.704444, test accuracy 0.518667, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 1600, training accuracy 0.676778, test accuracy 0.503333, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 1800, training accuracy 0.727667, test accuracy 0.512667, learning rate 0.01, max_train_acc 0.8637

step 3000, training accuracy 0.775556, test accuracy 0.572, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 3200, training accuracy 0.765333, test accuracy 0.567333, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 3400, training accuracy 0.762778, test accuracy 0.552, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 3600, training accuracy 0.736, test accuracy 0.558667, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 3800, training accuracy 0.746444, test accuracy 0.528667, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 4000, training accuracy 0.767778, test accuracy 0.573333, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 4200, training accuracy 0.777, test accuracy 0.565333, learning rate 0.01, max_train_acc 0.863778, max_test_acc 0.644667
step 4400, training accuracy 0.790667, test accuracy 0.58, learning rate 0.01, max_train_acc 0.863778, max_te

KeyboardInterrupt: 